In [1]:
import numpy as np
import pandas as pd
import glob, os, vcf, warnings, shutil, subprocess, re, sys
from Bio import Seq, SeqIO
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import scipy
from Bio import Seq, SeqIO, Entrez
from dna_features_viewer import BiopythonTranslator
# import panel as pn
# pn.extension('ipywidgets')
# pn.extension()

h37Rv_path = "/n/data1/hms/dbmi/farhat/Sanjana/H37Rv"
h37Rv_seq = SeqIO.read(os.path.join(h37Rv_path, "GCF_000195955.2_ASM19595v2_genomic.gbff"), "genbank")
h37Rv_genes = pd.read_csv(os.path.join(h37Rv_path, "mycobrowser_h37rv_genes_v4.csv"))
h37Rv_regions = pd.read_csv(os.path.join(h37Rv_path, "mycobrowser_h37rv_v4.csv"))

h37Rv_coords = pd.read_csv(os.path.join(h37Rv_path, "h37Rv_coords_to_gene.csv"))
h37Rv_coords_dict = dict(zip(h37Rv_coords["pos"].values, h37Rv_coords["region"].values))

# keep only the 727 patients with high quality WGS and matched patients
df_TRUST_pids_samples = pd.read_csv("/home/sak0914/MtbQuantCNN/analysis/TRUST/processed_data/combined_patient_WGS_data.csv").dropna(subset=['pid', 'Lineage']).reset_index(drop=True)
print(df_TRUST_pids_samples.pid.nunique(), df_TRUST_pids_samples.SampleID.nunique())

418 747


In [11]:
df_TRUST_pids_samples['Lineage'] = df_TRUST_pids_samples['Lineage'].astype(str)

In [26]:
mixed_samples = df_TRUST_pids_samples.query("F2 > 0.03").SampleID.unique()
len(mixed_samples)

30

In [29]:
add_samples = list(set(mixed_samples) - set(os.listdir(output_dir)))

In [37]:
df_isolates = pd.DataFrame(columns = ['SampleID', 'Percents'])
i = 0

for sample in os.listdir(output_dir):

    combined_percents_string = []
    
    for fName in glob.glob(f"{output_dir}/{sample}/*_strain.reads"):
        combined_percents_string.append(os.path.basename(fName).split('_strain.reads')[0])

    if len(combined_percents_string) >= 0:
        df_isolates.loc[i, :] = [sample, ','.join(combined_percents_string)]
        i += 1

len(df_isolates), len(os.listdir(output_dir))

(30, 30)

In [39]:
df_isolates.to_csv("./isolates_to_run.tsv", sep='\t', header=None, index=False)

In [7]:
df_TRUST_pids_samples.query("SampleID in ['MFS-350', 'MFS-353']")

,pid,Original_ID,SampleID,SampleLib,Pacbio,Comment,quality,Cov_Any_Mean,Cov_Unam_Perc,Perc_Reads_Mapped,...,fib4,prevtb_outcome,total_adherence,adherence_12week,first_dots_date,last_dots_date,F2,Coll2014,Freschi2020,Lineage
379,T0087,S0087-01,MFS-353,MFS-353_lib59077,NaN,NaN,1.0,422.31,1.0,0.9897,...,0.614105,2.0,0.983607,1.000000,1,178,0.415768,"4.1.2.1,4.8","4.2.1.1.1.1.1.1.i1,4.1.i1.1.1.1",4
400,T0072,S0072-01,MFS-350,MFS-350_lib59074,NaN,NaN,1.0,346.84,1.0,0.9899,...,0.285549,NaN,0.819549,0.833333,1,189,0.408194,"4.1.2.1,4.8","4.2.1.1.1.1.1.1.i1,4.1.i1.1.1.1",4


In [6]:
for sample in os.listdir(output_dir):
    try:
        result = pd.read_csv(f"{output_dir}/{sample}/result.txt", sep='\t', comment='I')
        print(sample, result['p-value'].values[0], result['proportions'].values[0])
    except:
        print(f"{sample} not yet finished")

MFS-134 0.0 0.87 0.13 
MFS-145 0.0 0.87 0.13 
MFS-198 0.0 0.85 0.15 
MFS-270 0.0 0.79 0.21 
MFS-313 0.0 0.88 0.12 
MFS-317 0.0 0.71 0.29 
MFS-336 0.0 0.52 0.48 
MFS-350 1.0 1.0
MFS-353 1.0 1.0
MFS-377 0.0 0.87 0.13 
MFS-39 0.0 0.86 0.14 
MFS-450 0.0 0.83 0.17 
MFS-456 0.0 0.86 0.14 
MFS-485 0.0 0.86 0.14 
MFS-486 0.0 0.85 0.15 
MFS-577 0.0 0.85 0.15 
MFS-608 0.0 0.85 0.15 
MFS-630 0.0 0.85 0.15 
MFS-631 0.0 0.74 0.26 
MFS-696 0.0 0.84 0.16 
MFS-835 0.0 0.86 0.14 
MFS-84 0.0 0.82 0.18 
MFS-85 0.0 0.76 0.24 


In [9]:
df_variants = pd.read_csv(f"../processed_data/phase_variation_regions/indels_HT.csv")

In [20]:
df_variants.merge(df_isolates, on='SampleID')[['SampleID', 'POS', 'REF', 'ALT', 'AF', 'F2', 'Lineage', 'Coll2014']].query("AF >= 0.05 & AF <= 0.75").sort_values("F2", ascending=False)

,SampleID,POS,REF,ALT,AF,F2,Lineage,Coll2014
1,MFS-336,2536628,G,GA,0.480952,0.458952,"2,4","4.1.1.1,2.2.1.1"
29,MFS-313,2536628,G,GA,0.472527,0.436810,"2,4","4.3.2.1,2.2.1.1"
28,MFS-313,854252,GC,G,0.404844,0.436810,"2,4","4.3.2.1,2.2.1.1"
30,MFS-317,854252,GC,G,0.626062,0.369685,"2,4","4.3.2.1,2.2.1.1"
31,MFS-317,1572680,G,GC,0.632530,0.369685,"2,4","4.3.2.1,2.2.1.1"
32,MFS-317,2536628,G,GA,0.616541,0.369685,"2,4","4.3.2.1,2.2.1.1"
19,MFS-631,2536628,G,GA,0.732558,0.314259,"2,3","2.2.1,3.1.1"
11,MFS-85,854252,GC,G,0.729299,0.259418,"2,4","2.2.1,4.1.1.3"
12,MFS-85,2536628,G,GA,0.646226,0.259418,"2,4","2.2.1,4.1.1.3"
17,MFS-630,36470,GCC,G,0.177083,0.147559,2,2.2.1
